In [15]:
# Imports
import pandas as pd
import numpy as np

# Jupyter pip install packages (note: %pip is a magic function: https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-pip)
%pip install xlrd
%pip install openpyxl
%pip install sqlalchemy
%pip install psycopg2-binary

from sqlalchemy import create_engine
from sqlalchemy.types import Float

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Import file and view example data
# Note: Based on data exploration, I added some converters to preserve incoming data for certain numeric types
filename = "GE-DMYR-2015-Final-Rates.xls"
df = pd.read_excel(filename, converters={'Zip': lambda x: str(x), 'CIP Code': lambda x: str(x)})

# All the cleaning from the exploration notebook

# Only keep needed columns
df = df[['Institution Name', 'City', 'State', 'Zip', 'Institution Type', 
         'CIP Code', 'CIP Name', 'Credential Level', 'Debt-to-Earnings Annual Rate', 
         'Mean  Annual Earnings From SSA', 'Median Annual Earnings from SSA']]

# Find the non-numeric data point (DROP FOR NOW)
# This could be done better
df[~df['Debt-to-Earnings Annual Rate'].apply(np.isreal)]   # 103 rows will be dropped :(
df['Debt-to-Earnings Annual Rate'] = pd.to_numeric(df['Debt-to-Earnings Annual Rate'], errors='coerce')

# Dop na
df.dropna(inplace=True)

# Create unique id field
df.reset_index(inplace=True)
df.rename(columns = {'index':'id'}, inplace=True)

# Rename columns
df.rename(columns={
    "Institution Name" : "institutionName",
    "City" : "city",
    "State" : "state",
    "Zip" : "zip",
    "Institution Type" : "institutionType",
    "CIP Code" : "cipCode",
    "CIP Name" : "cipName",
    "Credential Level" : "credentialLevel",
    "Debt-to-Earnings Annual Rate" : "debtToEarningsAnnualRate",
    "Mean  Annual Earnings From SSA" : "meanAnnualEarningsFromSsa",
    "Median Annual Earnings from SSA" : "medianAnnualEarningsfromSsa"
}, inplace=True)

,id,institutionName,city,state,zip,institutionType,cipCode,cipName,credentialLevel,debtToEarningsAnnualRate,meanAnnualEarningsFromSsa,medianAnnualEarningsfromSsa
0,0,CENTRAL ALABAMA COMMUNITY COLLEGE,ALEXANDER CITY,AL,350100000,PUBLIC 2 TO 3 YEARS,513901,LICENSED PRACTICAL/VOCATIONAL NURSE TRAINING.,01 - UNDERGRADUATE CERTIFICATE,2.50,32617,35061
1,1,CALHOUN COMMUNITY COLLEGE,TANNER,AL,356710000,PUBLIC 2 TO 3 YEARS,513901,LICENSED PRACTICAL/VOCATIONAL NURSE TRAINING.,01 - UNDERGRADUATE CERTIFICATE,0.78,26219,31071
2,2,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269,PUBLIC 2 TO 3 YEARS,120401,"COSMETOLOGY/COSMETOLOGIST, GENERAL.",01 - UNDERGRADUATE CERTIFICATE,0.00,12114,11605
3,3,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269,PUBLIC 2 TO 3 YEARS,480508,WELDING TECHNOLOGY/WELDER.,01 - UNDERGRADUATE CERTIFICATE,0.00,29104,29702
4,4,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269,PUBLIC 2 TO 3 YEARS,510909,SURGICAL TECHNOLOGY/TECHNOLOGIST.,01 - UNDERGRADUATE CERTIFICATE,0.00,21605,23598


In [18]:
# Write to table
engine = create_engine('<DB-STRING>')
df.to_sql('api_debttoearningsdata', engine, if_exists='append', index=False, 
          dtype={"debtToEarningsAnnualRate": Float(), 
               "meanAnnualEarningsFromSsa": Float(),
               "medianAnnualEarningsfromSsa": Float()})
